In [1]:
from faiss_module import load_and_vectorize,load_chunks_make_docdb
from model import setup_llm_pipeline
from fewshot_module import  fewshot_ex
from save_module import save
from seed_module import seed_everything
from utils_module import make_dict, extract_answer, format_docs
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

model_id =  "meta-llama/Meta-Llama-3.1-8B-Instruct"
i=2
train_db = load_chunks_make_docdb('./train_source', './train_faiss_db')
train_retriever = train_db.as_retriever(search_kwargs={'k': 3})
# train_dict = make_dict('train.csv')

test_db = load_chunks_make_docdb('./test_source', './test_faiss_db')
test_retriver = test_db.as_retriever(search_kwargs={'k': 3})
test_dict = make_dict('test.csv')

fewshot_db = load_and_vectorize('train.csv', './fewshot_faiss_db')

# llm = setup_llm_pipeline(model_id)
# Semantic Similarity Example Selector 설정


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading FAISS DB from: ./train_faiss_db
Loading FAISS DB from: ./test_faiss_db
Loading FAISS DB from: ./fewshot_faiss_db


In [4]:
def fewshot_ex(fewshot_db, buff, fewshot_num, train_retriever = None):
    template ="""<|start_header_id|>user<|end_header_id|>\n{Question}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n{Answer}<|eot_id|>\n\n"""

    example_selector = SemanticSimilarityExampleSelector(
        vectorstore=fewshot_db,
        k=fewshot_num,
    )
    ex = example_selector.select_examples({'Question': buff['Question']})
    fewshot_str = ""
    for i , entry in enumerate(ex):
        question = entry['Question']
        if train_retriever is not None:
            fewshot_prompt = PromptTemplate.from_template("""<|start_header_id|>system<|end_header_id|>
Given the following contexts about Question:\n{contents}"""+template)
            retrieved_docs = train_retriever.invoke(question)
            ex[i]['contents'] = format_docs(retrieved_docs)
        else:
            fewshot_prompt = PromptTemplate.from_template(template)
        fewshot_str += fewshot_prompt.format(**ex[i])
    return fewshot_str

print(fewshot_ex(fewshot_db, test_dict[0], 3, train_retriever))

<|start_header_id|>system<|end_header_id|>
Given the following contexts about Question:
Document 1
사   업   명
창업사업화지원 (5132-302)
1.사업 코드 정보
구분 회계 소관 실국 (기관 )계정 분야 부문
코드
일반회계중소벤처기
업부창업정책관110 118
명칭산업·중소기업 
및 에너지창업 및 벤처
구분 프로그램 단위사업 세부사업
코드 5100 5132 302
명칭 창업환경조성 창업활성화지원 창업사업화지원
2.사업 지원 형태 및 지원율 (최소한 한 개는 반드시 선택하시오 .해당사항에 ○표시 )
직접 출자 출연 보조 융자 국고보조율 (%) 융자율 (%)
○ ○○
3.예산 총괄표
(단위 :백만원 ,%)
사업명2022년
결산2023년 2024년 증감
(B-A) 본예산 (A) 추경 확정 (B) (B-A)/ A
창업사업화지원 432,824 388,910 388,910 377,656△11,254△2.9
4.사업목적
ㅇ (창업사업화지원 )창업기업의 성장단계별 (창업준비 ,창업초기 및 도약기 ),초격차 분야별 ,
글로벌화 지원체계를 구축‧
운영하여 혁신 기술창업을 활성화하고 창업기업 성장 및 생
존율 제고<|eot_id|>
Document 2
(100.0) (71.0) (19.1) (7.9) (10.0)
2023831,321 562,202 178,042 94,953 91,078
(100.0) (67.6) (21.4) (11.4) (11.0)
2024892,667 619,904 182,517 100,149 90,247
(100.0) (69.4) (20.4) (11.2) (10.1)
주: 2022년까지 결산, 2023~2024년 예산기준
자료: 디지털예산회계시스템3.
국고보조금<|eot_id|>
Document 3
주요 재정통계●
Ⅰ.
18▸(사업성기금) 2024년 예산 기준, 사업성기금 운용규모는 총지출 기준 81.2조원이며 49개가 운영
∙기금별 운용규모는 주택도시기금이 37.2조원(45

In [2]:
template ="""<|start_header_id|>user<|end_header_id|>\n{Question}<|eot_id|>\n\n<|start_header_id|>assistant<|end_header_id|>\n{Answer}<|eot_id|>"""
example_prompt = PromptTemplate.from_template(template)
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=fewshot_db,
    k=2,
)


In [3]:
fewshot_prompt = PromptTemplate.from_template("""<|start_header_id|>system<|end_header_id|>
Given the following contexts about Question:\n{contents}"""+template)


In [4]:
ex = example_selector.select_examples({'Question': test_dict[i]['Question']})
fewshot_str = ""
for i , entry in enumerate(ex):
    question = entry['Question']
    retrieved_docs = train_retriever.invoke(question)
    ex[i]['contents'] = format_docs(retrieved_docs)
    fewshot_str += fewshot_prompt.format(**ex[i])


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [5]:
print(fewshot_str)


<|start_header_id|>system<|end_header_id|>
Given the following contexts about Question:
존율 제고
5.사업근거 및 추진경위
ㅇ 법령상 근거 :「중소기업창업 지원법」  제10조(창업 활성화 지원사업의 추진 등),「기술의 이전 
및 사업화 촉진에 관한 법률」제15조(기술이전ㆍ사업화 촉진사업의 추진)

5.사업근거 및 추진경위
①법령상 근거 :｢
국민 평생 직업능력개발법｣
제12조,제15조,제17조 및 제18조
제12조(직업능력개발훈련 지원 등)①국가와 지방자치단체는 국민의 고용창출 ,고용촉진 및 
고용안정을 위하여 직업능력개발훈련을 실시하거나 직업능력개발훈련을 받는 사람에게 비용을 지
원할 수 있다 .이 경우 제3조제 4항 각 호에 해당하는 사람에 대하여는 우선적으로 지원될 수 
있도록 하여야 한다 .
②제1항에 따라 실시하는 직업능력개발훈련의 대상 ,훈련과정의 요건 ,훈련수당 ,그 밖에 
직업능력개발훈련에 필요한 사항은 대통령령으로 정한다 .
제15조(국가기간ㆍ전략산업직종에 대한 직업능력개발훈련의 실시 )①국가와 지방자치단체는 
다음 각 호의 직종 (이하 “국가기간ㆍ전략산업직종 ”이라 한다 )에 대한 원활한 인력수급을 
위하여 필요한 직업능력개발훈련을 실시할 수 있다 .
1.국가경제의 기간 (基幹 )이 되는 산업 중 인력이 부족한 직종

6.주요내용
ㅇ 사업기간 :‘09~
ㅇ 사업규모 :377,656백만원 /창업기업 3,500개사 지원
ㅇ 사업시행방법 :직접 ,보조 ,출연
ㅇ 사업시행주체 :중소벤처기업부 ,창업진흥원 등
ㅇ 사업 수혜자 :창업기업
ㅇ 보조 ,융자 ,출연 ,출자 등의 경우 보조·융자 등 지원 비율 및 법적근거
피보조 ·피출연 등 기관명 지원 비율 (%) 보조율 법적근거 (해당 조항 )
창업진흥원 70-100% 중소기업창업 지원법 제10조,제32조,제51조
7.사업 집행절차
<사업추진 절차 >
①사업공고
➜②신청
➜③선정평가
➜④협약체결
중소벤처기업부 창업기업→전담기관 평가위원회 